In [ ]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

import spacy
try:
    nlp = spacy.load("ru_core_news_sm")
except OSError:
    print("Установите русскую модель Spacy: python -m spacy download ru_core_news_sm")
    nlp = None

import joblib
from rapidfuzz import process, fuzz
from typing import Dict, List, Tuple, Any, Optional

class TextPreprocessor:
    """Класс для предварительной обработки текста"""
    
    def __init__(self):
        self.russian_stop_words = self._get_russian_stop_words()
    
    def _get_russian_stop_words(self) -> set:
        """Русские стоп-слова"""
        return {
            'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 
            'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 
            'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 
            'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 
            'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 
            'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 
            'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 
            'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 
            'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'ж', 'тогда', 'кто', 
            'этот', 'того', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 
            'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 
            'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 
            'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 
            'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 
            'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 
            'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 
            'всю', 'между'
        }
    
    def preprocess_text(self, text: str) -> str:
        """Основная предобработка текста"""
        if pd.isna(text):
            return ""
        
        text = str(text).lower()
        
        # Удаляем специальные символы, но сохраняем числа и единицы измерения
        text = re.sub(r'[^\w\s\d\.\,\-\+\/\±]', ' ', text)
        
        # Заменяем множественные пробелы на один
        text = re.sub(r'\s+', ' ', text)
        
        return text.strip()
    
    def extract_units_and_numbers(self, text: str) -> Dict[str, List[Tuple]]:
        """Извлечение чисел с единицами измерения"""
        patterns = {
            'length': r'(\d+[.,]?\d*)\s*(мм|см|м|метр|сантиметр|миллиметр)',
            'weight': r'(\d+[.,]?\d*)\s*(г|кг|тонн|грамм|килограмм)',
            'voltage': r'(\d+[.,]?\d*)\s*(в|вольт|кв|киловольт)',
            'current': r'(\d+[.,]?\d*)\s*(а|ампер|ка|килоампер)',
            'power': r'(\d+[.,]?\d*)\s*(вт|квт|ватт|киловатт)',
            'diameter': r'(\d+[.,]?\d*)\s*(мм|см|м)',
            'temperature': r'(\d+[.,]?\d*)\s*(°c|°f|с|градус)',
        }
        
        results = {}
        for unit_type, pattern in patterns.items():
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                results[unit_type] = matches
        return results

class CharacteristicExtractor:
    """Класс для извлечения характеристик из текста"""
    
    def __init__(self):
        self.preprocessor = TextPreprocessor()
        self.patterns = self._initialize_patterns()
    
    def _initialize_patterns(self) -> Dict[str, List[str]]:
        """Инициализация паттернов для извлечения характеристик"""
        return {
            'сечение': ['сечение', 'сеч', 'площадь', 'мм2', 'мм²'],
            'длина': ['длина', 'длинна', 'метраж'],
            'напряжение': ['напряжение', 'вольтаж', 'u', 'v'],
            'ток': ['ток', 'сила тока', 'ампераж', 'i', 'a'],
            'мощность': ['мощность', 'p', 'w'],
            'диаметр': ['диаметр', 'диам', 'ø'],
            'вес': ['вес', 'масса'],
            'цвет': ['цвет', 'окраска'],
            'материал': ['материал', 'изготовлен', 'сделан'],
            'производитель': ['производитель', 'бренд', 'фирма', 'марка'],
            'температура': ['температура', 'темп', 't°'],
            'защита': ['ip', 'защита', 'степень защиты'],
        }
    
    def extract_with_patterns(self, text: str, characteristics: List[str]) -> Dict[str, str]:
        """Извлечение характеристик с помощью паттернов"""
        results = {}
        text_clean = self.preprocessor.preprocess_text(text)
        
        for char in characteristics:
            if char not in self.patterns:
                continue
                
            patterns = self.patterns[char]
            found = False
            
            for pattern in patterns:
                # Паттерн: характеристика: значение
                match1 = re.search(rf'{pattern}[:\s]+([^,.;]+?)(?=[,.;]|$)', text_clean)
                # Паттерн: характеристика - значение
                match2 = re.search(rf'{pattern}[\s\-]+([^,.;]+?)(?=[,.;]|$)', text_clean)
                # Паттерн: значение характеристика
                match3 = re.search(rf'([^,.;]+?)\s+{pattern}(?=[,.;]|$)', text_clean)
                
                for match in [match1, match2, match3]:
                    if match:
                        value = match.group(1).strip()
                        if value and len(value) < 50:  # Фильтр слишком длинных значений
                            results[char] = value
                            found = True
                            break
                if found:
                    break
        return results
    
    def extract_with_fuzzy_matching(self, text: str, characteristics: List[str], 
                                  threshold: int = 75) -> Dict[str, str]:
        """Извлечение с помощью нечеткого сопоставления"""
        results = {}
        text_clean = self.preprocessor.preprocess_text(text)
        words = text_clean.split()
        
        for char in characteristics:
            if char not in self.patterns:
                continue
                
            patterns = self.patterns[char]
            best_score = 0
            best_value = None
            
            for pattern in patterns:
                # Ищем похожие слова в тексте
                for i, word in enumerate(words):
                    score = fuzz.partial_ratio(pattern, word)
                    if score > best_score and score >= threshold:
                        best_score = score
                        # Пытаемся извлечь значение после найденного слова
                        if i + 1 < len(words):
                            best_value = words[i + 1]
            
            if best_value:
                results[char] = best_value
                
        return results
    
    def extract_advanced(self, text: str, characteristics: List[str]) -> Dict[str, str]:
        """Комплексное извлечение характеристик"""
        # 1. Паттернный анализ
        pattern_results = self.extract_with_patterns(text, characteristics)
        
        # 2. Нечеткое сопоставление
        fuzzy_results = self.extract_with_fuzzy_matching(text, characteristics)
        
        # 3. Объединяем результаты
        results = {**fuzzy_results, **pattern_results}  # pattern_results имеет приоритет
        
        # 4. Постобработка значений
        cleaned_results = {}
        for char, value in results.items():
            # Очистка значения от лишних слов
            value_clean = re.sub(r'^(и|или|на|в|с|по)\s+', '', value)
            value_clean = value_clean.strip()
            if value_clean:
                cleaned_results[char] = value_clean
                
        return cleaned_results

class NomenclatureClassifier:
    """Классификатор номенклатуры по классам"""
    
    def __init__(self):
        self.model = None
        self.vectorizer = TfidfVectorizer(max_features=2000, ngram_range=(1, 2))
        self.label_encoder = LabelEncoder()
        self.is_trained = False
    
    def train(self, descriptions: List[str], classes: List[str]):
        """Обучение классификатора"""
        # Предобработка текстов
        preprocessor = TextPreprocessor()
        cleaned_descriptions = [preprocessor.preprocess_text(desc) for desc in descriptions]
        
        # Векторизация
        X = self.vectorizer.fit_transform(cleaned_descriptions)
        y = self.label_encoder.fit_transform(classes)
        
        # Обучение модели
        self.model = RandomForestClassifier(
            n_estimators=100, 
            random_state=42,
            class_weight='balanced'
        )
        self.model.fit(X, y)
        self.is_trained = True
        
        print(f"Классификатор обучен на {len(descriptions)} примерах")
        print(f"Количество классов: {len(self.label_encoder.classes_)}")
    
    def predict(self, description: str) -> str:
        """Предсказание класса для описания"""
        if not self.is_trained:
            raise ValueError("Классификатор не обучен")
        
        cleaned_desc = TextPreprocessor().preprocess_text(description)
        X = self.vectorizer.transform([cleaned_desc])
        y_pred = self.model.predict(X)
        return self.label_encoder.inverse_transform(y_pred)[0]
    
    def predict_proba(self, description: str) -> Dict[str, float]:
        """Предсказание с вероятностями"""
        if not self.is_trained:
            raise ValueError("Классификатор не обучен")
        
        cleaned_desc = TextPreprocessor().preprocess_text(description)
        X = self.vectorizer.transform([cleaned_desc])
        probabilities = self.model.predict_proba(X)[0]
        
        return {
            class_name: prob 
            for class_name, prob in zip(self.label_encoder.classes_, probabilities)
        }

class NomenclatureProcessor:
    """Основной класс для обработки номенклатуры"""
    
    def __init__(self):
        self.classifier = NomenclatureClassifier()
        self.extractor = CharacteristicExtractor()
        self.class_characteristics = {}  # class -> list of characteristics
        self.is_trained = False
    
    def load_training_data(self, normalized_data: pd.DataFrame, 
                         characteristics_data: pd.DataFrame):
        """Загрузка обучающих данных"""
        
        # Анализ характеристик по классам
        self.class_characteristics = {}
        for _, row in characteristics_data.iterrows():
            class_name = row['class']
            characteristic = row['characteristic']
            
            if class_name not in self.class_characteristics:
                self.class_characteristics[class_name] = []
            
            if characteristic not in self.class_characteristics[class_name]:
                self.class_characteristics[class_name].append(characteristic)
        
        # Обучение классификатора классов
        descriptions = normalized_data['description'].tolist()
        classes = normalized_data['class'].tolist()
        
        self.classifier.train(descriptions, classes)
        self.is_trained = True
        
        print("Данные загружены и модели обучены")
        print("Доступные классы:", list(self.class_characteristics.keys()))
    
    def process_single_item(self, class_name: Optional[str], 
                          nomenclature: str, description: str) -> Dict[str, Any]:
        """Обработка одной позиции номенклатуры"""
        
        # Если класс не указан, пытаемся предсказать
        if not class_name or pd.isna(class_name):
            if self.is_trained:
                class_name = self.classifier.predict(description)
                predicted = True
            else:
                class_name = "unknown"
                predicted = True
        else:
            predicted = False
        
        # Получаем характеристики для класса
        characteristics_list = self.class_characteristics.get(class_name, [])
        
        # Извлекаем характеристики
        if characteristics_list:
            extracted_chars = self.extractor.extract_advanced(description, characteristics_list)
        else:
            extracted_chars = {}
        
        return {
            'nomenclature': nomenclature,
            'class': class_name,
            'class_predicted': predicted,
            'description': description,
            'characteristics': extracted_chars,
            'characteristics_list': characteristics_list
        }
    
    def process_batch(self, unnormalized_data: pd.DataFrame) -> pd.DataFrame:
        """Пакетная обработка ненормализованных данных"""
        
        results = []
        
        for _, row in unnormalized_data.iterrows():
            class_name = row.get('class')
            nomenclature = row.get('nomenclature')
            description = row.get('description')
            
            if pd.isna(description):
                print(f"Пропущено: отсутствует описание для {nomenclature}")
                continue
            
            result = self.process_single_item(class_name, nomenclature, description)
            results.append(result)
        
        # Создаем DataFrame с результатами
        result_df = pd.DataFrame(results)
        
        # Разворачиваем характеристики в отдельные колонки
        characteristics_df = result_df['characteristics'].apply(pd.Series)
        final_df = pd.concat([result_df.drop('characteristics', axis=1), characteristics_df], axis=1)
        
        return final_df
    
    def save_model(self, filepath: str):
        """Сохранение модели"""
        if not self.is_trained:
            raise ValueError("Модель не обучена")
        
        model_data = {
            'classifier': self.classifier,
            'extractor': self.extractor,
            'class_characteristics': self.class_characteristics,
            'is_trained': self.is_trained
        }
        
        joblib.dump(model_data, filepath)
        print(f"Модель сохранена в {filepath}")
    
    @classmethod
    def load_model(cls, filepath: str) -> 'NomenclatureProcessor':
        """Загрузка модели"""
        model_data = joblib.load(filepath)
        
        processor = cls()
        processor.classifier = model_data['classifier']
        processor.extractor = model_data['extractor']
        processor.class_characteristics = model_data['class_characteristics']
        processor.is_trained = model_data['is_trained']
        
        print(f"Модель загружена из {filepath}")
        return processor

# Пример использования
def create_sample_data() -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Создание примеров данных для демонстрации"""
    
    # Нормализованные данные для обучения
    normalized_data = pd.DataFrame({
        'nomenclature': [
            'Кабель ВВГ 3х2.5',
            'Труба ПНД 32мм',
            'Автомат 16А',
            'Светильник LED 18W',
            'Розетка Schneider Electric',
            'Кабель ВВГнг 3х1.5',
            'Труба ПВХ 20мм',
            'Автомат 25А',
            'Светильник накладной',
            'Выключатель Legrand'
        ],
        'class': ['кабель', 'труба', 'автомат', 'светильник', 'розетка',
                 'кабель', 'труба', 'автомат', 'светильник', 'розетка'],
        'description': [
            'Кабель силовой ВВГ 3х2.5 мм², длина 100м, напряжение 660В',
            'Труба полиэтиленовая ПНД диаметр 32 мм, длина 2м, давление 10атм',
            'Автоматический выключатель 16А, характеристика C, полюсов 1',
            'Светильник светодиодный LED мощность 18W, цвет белый, IP65',
            'Розетка электрическая Schneider Electric, 220В, тип C',
            'Кабель ВВГнг 3х1.5 мм² негорючий, напряжение 0.66кВ',
            'Труба ПВХ диаметр 20 мм для электропроводки, длина 3м',
            'Автомат двухполюсный 25А, характеристика B',
            'Светильник накладной для помещений, мощность 36W',
            'Выключатель клавишный Legrand, 10А, белый'
        ]
    })
    
    # Характеристики для каждого класса
    characteristics_data = pd.DataFrame({
        'class': ['кабель', 'кабель', 'кабель', 'труба', 'труба', 'труба', 
                 'автомат', 'автомат', 'светильник', 'светильник', 'розетка', 'розетка'],
        'characteristic': ['сечение', 'длина', 'напряжение', 'диаметр', 'длина', 'материал',
                          'ток', 'характеристика', 'мощность', 'цвет', 'напряжение', 'производитель']
    })
    
    # Ненормализованные данные для обработки
    unnormalized_data = pd.DataFrame({
        'class': [None, 'труба', None, 'светильник'],
        'nomenclature': [
            'Провод медный многожильный',
            'Труба пластиковая для кабеля',
            'Выключатель автоматический 25А',
            'Лампа светодиодная уличная'
        ],
        'description': [
            'Провод медный многожильный сечение 4мм² длина 50м напряжение 380В',
            'Труба пластиковая диаметр 40мм длина 3м давление 8атм',
            'Автоматический выключатель номинальный ток 25А характеристика B',
            'Светильник светодиодный мощность 30W степень защиты IP67 цвет черный'
        ]
    })
    
    return normalized_data, characteristics_data, unnormalized_data

def main():
    """Основная функция демонстрации"""
    
    print("=== СИСТЕМА ПАРСИНГА ХАРАКТЕРИСТИК НОМЕНКЛАТУРЫ ===\n")
    
    # Создаем пример данных
    normalized_data, characteristics_data, unnormalized_data = create_sample_data()
    
    print("1. ДАННЫЕ ДЛЯ ОБУЧЕНИЯ:")
    print("Нормализованные данные:")
    print(normalized_data)
    print("\nХарактеристики по классам:")
    print(characteristics_data)
    print("\nДанные для обработки:")
    print(unnormalized_data)
    
    # Инициализация и обучение процессора
    print("\n2. ОБУЧЕНИЕ МОДЕЛЕЙ...")
    processor = NomenclatureProcessor()
    processor.load_training_data(normalized_data, characteristics_data)
    
    # Обработка ненормализованных данных
    print("\n3. ОБРАБОТКА ДАННЫХ...")
    results = processor.process_batch(unnormalized_data)
    
    print("\n4. РЕЗУЛЬТАТЫ ОБРАБОТКИ:")
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    print(results)
    
    # Сохранение модели
    print("\n5. СОХРАНЕНИЕ МОДЕЛИ...")
    processor.save_model('nomenclature_processor.pkl')
    
    # Демонстрация загрузки и использования модели
    print("\n6. ТЕСТ ЗАГРУЗКИ МОДЕЛИ...")
    loaded_processor = NomenclatureProcessor.load_model('nomenclature_processor.pkl')
    
    # Тестирование на новых данных
    test_data = pd.DataFrame({
        'class': [None],
        'nomenclature': ['Кабель силовой медный'],
        'description': ['Кабель ВВГнг 3х6мм² длина 100м напряжение 0.66кВ']
    })
    
    test_results = loaded_processor.process_batch(test_data)
    print("\nРезультаты теста:")
    print(test_results)
    
    return processor, results

if __name__ == "__main__":
    processor, results = main()

In [ ]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from rapidfuzz import process, fuzz
import joblib
from typing import Dict, List, Tuple, Any, Optional
from collections import defaultdict

class NomenclatureParser:
    """Парсер характеристик номенклатуры на основе нормализованных данных"""
    
    def __init__(self):
        self.category_classifier = None
        self.vectorizer = None
        self.label_encoder = LabelEncoder()
        self.category_characteristics = defaultdict(list)  # category -> list of characteristics
        self.characteristic_patterns = defaultdict(dict)   # (category, characteristic) -> patterns
        self.value_patterns = defaultdict(list)           # characteristic -> common value patterns
        self.is_trained = False
        
    def load_normalized_data(self, file_path: str) -> pd.DataFrame:
        """Загрузка нормализованных данных из файла"""
        try:
            # Пробуем разные разделители
            df = pd.read_csv(file_path, sep='\t', encoding='utf-8')
            if df.shape[1] == 1:
                df = pd.read_csv(file_path, sep=',', encoding='utf-8')
        except:
            df = pd.read_csv(file_path, encoding='utf-8', sep=None, engine='python')
        
        # Проверяем наличие необходимых колонок
        required_columns = ['nomenclature', 'category', 'description', 'characteristic', 'value']
        missing_columns = [col for col in required_columns if col not in df.columns]
        
        if missing_columns:
            raise ValueError(f"Отсутствуют обязательные колонки: {missing_columns}")
        
        print(f"Загружено {len(df)} записей нормализованных данных")
        print(f"Количество категорий: {df['category'].nunique()}")
        print(f"Количество характеристик: {df['characteristic'].nunique()}")
        
        return df
    
    def preprocess_text(self, text: str) -> str:
        """Предобработка текста"""
        if pd.isna(text):
            return ""
        
        text = str(text).lower()
        # Удаляем специальные символы, но сохраняем числа и единицы измерения
        text = re.sub(r'[^\w\s\d\.\,\-\+\/\±]', ' ', text)
        # Заменяем множественные пробелы на один
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    
    def analyze_characteristics(self, normalized_data: pd.DataFrame):
        """Анализ характеристик и их значений из нормализованных данных"""
        
        # Группируем по категориям и характеристикам
        for category in normalized_data['category'].unique():
            category_data = normalized_data[normalized_data['category'] == category]
            
            for char_name in category_data['characteristic'].unique():
                char_data = category_data[category_data['characteristic'] == char_name]
                
                # Добавляем характеристику в список для категории
                if char_name not in self.category_characteristics[category]:
                    self.category_characteristics[category].append(char_name)
                
                # Анализируем значения характеристики для извлечения паттернов
                values = char_data['value'].dropna().unique()
                
                for value in values:
                    value_str = str(value).lower()
                    
                    # Извлекаем паттерны из значений
                    patterns = self._extract_patterns_from_value(value_str)
                    for pattern in patterns:
                        if pattern not in self.value_patterns[char_name]:
                            self.value_patterns[char_name].append(pattern)
        
        print("Анализ характеристик завершен:")
        for category, chars in self.category_characteristics.items():
            print(f"  {category}: {len(chars)} характеристик")
    
    def _extract_patterns_from_value(self, value: str) -> List[str]:
        """Извлечение паттернов из значений характеристик"""
        patterns = []
        
        # Числовые паттерны
        number_patterns = [
            r'\d+[.,]?\d*',  # числа с плавающей точкой
            r'\d+',          # целые числа
        ]
        
        # Текстовые паттерны
        text_patterns = [
            r'[a-zа-я]+',    # слова
            r'[a-zа-я]+\s+[a-zа-я]+',  # словосочетания
        ]
        
        # Специальные паттерны (коды, модели)
        special_patterns = [
            r'[a-zа-я]\d+',           # буква + цифры
            r'\d+[a-zа-я]',           # цифры + буква
            r'[a-zа-я]\d+[a-zа-я]',   # буква + цифры + буква
            r'\d+-\d+',               # числа через дефис
            r'[a-zа-я]+-\d+',         # буквы-числа
        ]
        
        # Проверяем каждый паттерн
        all_patterns = number_patterns + text_patterns + special_patterns
        
        for pattern in all_patterns:
            matches = re.findall(pattern, value, re.IGNORECASE)
            for match in matches:
                if len(match) > 1 and match not in patterns:
                    patterns.append(match)
        
        return patterns
    
    def train_category_classifier(self, normalized_data: pd.DataFrame):
        """Обучение классификатора категорий"""
        
        # Создаем уникальный набор описаний для обучения
        unique_data = normalized_data.drop_duplicates(subset=['nomenclature', 'category', 'description'])
        
        if len(unique_data) == 0:
            raise ValueError("Недостаточно данных для обучения классификатора")
        
        descriptions = unique_data['description'].apply(self.preprocess_text).tolist()
        categories = unique_data['category'].tolist()
        
        # Кодируем категории
        categories_encoded = self.label_encoder.fit_transform(categories)
        
        # Векторизация текста
        self.vectorizer = TfidfVectorizer(
            max_features=1000,
            ngram_range=(1, 2),
            stop_words=None
        )
        
        X = self.vectorizer.fit_transform(descriptions)
        
        # Обучение модели
        self.category_classifier = RandomForestClassifier(
            n_estimators=100,
            random_state=42,
            class_weight='balanced'
        )
        
        self.category_classifier.fit(X, categories_encoded)
        
        print(f"Классификатор категорий обучен на {len(descriptions)} примерах")
        print(f"Количество категорий: {len(self.label_encoder.classes_)}")
    
    def predict_category(self, description: str) -> str:
        """Предсказание категории для описания"""
        if self.category_classifier is None:
            raise ValueError("Классификатор категорий не обучен")
        
        text = self.preprocess_text(description)
        X = self.vectorizer.transform([text])
        y_pred = self.category_classifier.predict(X)
        return self.label_encoder.inverse_transform(y_pred)[0]
    
    def extract_characteristics(self, category: str, description: str) -> Dict[str, str]:
        """Извлечение характеристик для указанной категории"""
        if category not in self.category_characteristics:
            return {}
        
        characteristics = {}
        text = self.preprocess_text(description)
        
        for char_name in self.category_characteristics[category]:
            value = self._extract_single_characteristic(char_name, text)
            if value:
                characteristics[char_name] = value
        
        return characteristics
    
    def _extract_single_characteristic(self, char_name: str, text: str) -> Optional[str]:
        """Извлечение значения одной характеристики"""
        
        # Паттерны для разных типов характеристик
        patterns = {
            'вид продукции': self._extract_product_type,
            'тип горелки': self._extract_burner_type,
            'тип самоспасателя': self._extract_respirator_type,
            'модель': self._extract_model,
        }
        
        # Используем специализированный метод если есть, иначе общий
        if char_name.lower() in patterns:
            return patterns[char_name.lower()](text)
        else:
            return self._extract_general_characteristic(char_name, text)
    
    def _extract_product_type(self, text: str) -> Optional[str]:
        """Извлечение вида продукции"""
        product_types = ['горелка', 'самоспасатель']
        
        for product_type in product_types:
            if product_type in text:
                return product_type.capitalize()
        
        return None
    
    def _extract_burner_type(self, text: str) -> Optional[str]:
        """Извлечение типа горелки"""
        burner_types = ['газовая', 'электрическая']
        
        for burner_type in burner_types:
            if burner_type in text:
                return burner_type
        
        # Ищем по ключевым словам
        if 'газ' in text:
            return 'газовая'
        elif 'электр' in text:
            return 'электрическая'
        
        return None
    
    def _extract_respirator_type(self, text: str) -> Optional[str]:
        """Извлечение типа самоспасателя"""
        respirator_types = ['изолирующий', 'фильтрующий']
        
        for resp_type in respirator_types:
            if resp_type in text:
                return resp_type
        
        return None
    
    def _extract_model(self, text: str) -> Optional[str]:
        """Извлечение модели/артикула"""
        # Паттерны для моделей
        model_patterns = [
            r'[a-zа-я]{2,}\s*[\-\s]*[a-zа-я]*\s*\d+[\-\s]*\d*[a-zа-я]*',  # ЗСУ-ПИ-38-350
            r'[a-zа-я]+\s*\d+[a-zа-я]*',                                  # R93A
            r'[a-zа-я]+\s*\d+[\s\-]*[a-zа-я]*\s*\d*',                     # СПИ-20
            r'[a-zа-я]+\s*\d+[a-zа-я]+\s*\d*',                           # ЗЕВС 30У
        ]
        
        for pattern in model_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                # Берем самую длинную найденную модель (обычно это полное обозначение)
                best_match = max(matches, key=len)
                return best_match.upper()
        
        return None
    
    def _extract_general_characteristic(self, char_name: str, text: str) -> Optional[str]:
        """Общий метод извлечения характеристик"""
        # Используем нечеткое сравнение для поиска значений
        if char_name in self.value_patterns:
            for pattern in self.value_patterns[char_name]:
                if pattern in text:
                    return pattern
        
        return None
    
    def train(self, normalized_data: pd.DataFrame):
        """Полное обучение модели на нормализованных данных"""
        print("=== ОБУЧЕНИЕ МОДЕЛИ ===")
        
        # Анализ характеристик
        self.analyze_characteristics(normalized_data)
        
        # Обучение классификатора категорий
        self.train_category_classifier(normalized_data)
        
        self.is_trained = True
        print("Модель успешно обучена")
    
    def process_unnormalized_data(self, unnormalized_data: pd.DataFrame) -> pd.DataFrame:
        """Обработка ненормализованных данных"""
        if not self.is_trained:
            raise ValueError("Модель не обучена. Сначала выполните обучение.")
        
        results = []
        
        for _, row in unnormalized_data.iterrows():
            nomenclature = row.get('nomenclature', '')
            category = row.get('category')
            description = row.get('description', '')
            
            # Если категория не указана, предсказываем её
            if pd.isna(category) or not category:
                try:
                    category = self.predict_category(description)
                    predicted_category = True
                except:
                    category = "unknown"
                    predicted_category = True
            else:
                predicted_category = False
            
            # Извлекаем характеристики
            characteristics = self.extract_characteristics(category, description)
            
            # Создаем результат
            result_row = {
                'nomenclature': nomenclature,
                'category': category,
                'category_predicted': predicted_category,
                'description': description,
                **characteristics
            }
            
            results.append(result_row)
        
        return pd.DataFrame(results)
    
    def save_model(self, filepath: str):
        """Сохранение модели"""
        if not self.is_trained:
            raise ValueError("Модель не обучена")
        
        model_data = {
            'category_classifier': self.category_classifier,
            'vectorizer': self.vectorizer,
            'label_encoder': self.label_encoder,
            'category_characteristics': dict(self.category_characteristics),
            'value_patterns': dict(self.value_patterns),
            'is_trained': self.is_trained
        }
        
        joblib.dump(model_data, filepath)
        print(f"Модель сохранена в {filepath}")
    
    @classmethod
    def load_model(cls, filepath: str) -> 'NomenclatureParser':
        """Загрузка модели"""
        model_data = joblib.load(filepath)
        
        parser = cls()
        parser.category_classifier = model_data['category_classifier']
        parser.vectorizer = model_data['vectorizer']
        parser.label_encoder = model_data['label_encoder']
        parser.category_characteristics = defaultdict(list, model_data['category_characteristics'])
        parser.value_patterns = defaultdict(list, model_data['value_patterns'])
        parser.is_trained = model_data['is_trained']
        
        print(f"Модель загружена из {filepath}")
        return parser

# Функции для работы с данными
def create_sample_unnormalized_data() -> pd.DataFrame:
    """Создание примера ненормализованных данных"""
    return pd.DataFrame({
        'nomenclature': ['1234', '5678', '91011', '121314'],
        'category': ['Горелки теплотехнические', None, 'Самоспасатели', None],
        'description': [
            'Горелка газовая ЗСУ-ПИ-38-350-IP65 для промышленного использования',
            'Электрическая горелка R93A M.PR.S.RU.A.8.50 с системой контроля',
            'Самоспасатель изолирующий СПИ-20 ТР ТС 019/2011',
            'Фильтрующий самоспасатель Parat 3100 ТР ТС 019/2011'
        ]
    })

def main():
    """Основная функция демонстрации"""
    print("=== СИСТЕМА ПАРСИНГА ХАРАКТЕРИСТИК НОМЕНКЛАТУРЫ ===\n")
    
    # Инициализация парсера
    parser = NomenclatureParser()
    
    try:
        # Загрузка нормализованных данных
        print("1. ЗАГРУЗКА НОРМАЛИЗОВАННЫХ ДАННЫХ...")
        normalized_data = parser.load_normalized_data('normalized.txt')
        print("Первые 5 записей нормализованных данных:")
        print(normalized_data.head())
        
        # Обучение модели
        print("\n2. ОБУЧЕНИЕ МОДЕЛИ...")
        parser.train(normalized_data)
        
        # Создание тестовых данных
        print("\n3. ПОДГОТОВКА ТЕСТОВЫХ ДАННЫХ...")
        unnormalized_data = create_sample_unnormalized_data()
        print("Данные для обработки:")
        print(unnormalized_data)
        
        # Обработка данных
        print("\n4. ОБРАБОТКА ДАННЫХ...")
        results = parser.process_unnormalized_data(unnormalized_data)
        
        print("\n5. РЕЗУЛЬТАТЫ ОБРАБОТКИ:")
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 1000)
        print(results)
        
        # Сохранение модели
        print("\n6. СОХРАНЕНИЕ МОДЕЛИ...")
        parser.save_model('nomenclature_parser.pkl')
        
        # Демонстрация загрузки модели
        print("\n7. ТЕСТ ЗАГРУЗКИ МОДЕЛИ...")
        loaded_parser = NomenclatureParser.load_model('nomenclature_parser.pkl')
        
        # Тестирование на новых данных
        test_data = pd.DataFrame({
            'nomenclature': ['9999'],
            'category': [None],
            'description': ['Станция радиолокационный ECAT2 252/12/MK/VM2 Sperry Marine']
        })
        
        test_results = loaded_parser.process_unnormalized_data(test_data)
        print("\nРезультаты теста новой модели:")
        print(test_results)
        
        return parser, results
        
    except Exception as e:
        print(f"Ошибка: {e}")
        # Создаем демонстрационные данные если файл не найден
        print("\nСоздание демонстрационных данных...")
        return create_demo_example()

def create_demo_example():
    """Создание демонстрационного примера если файл не найден"""
    # Создаем нормализованные данные на основе предоставленного примера
    normalized_data = pd.DataFrame({
        'nomenclature': ['1267', '1267', '1267', '1324', '1324', '1324', 
                        '2356', '2356', '2356', '4325', '4325', '4325'],
        'category': ['Горелки теплотехнические'] * 6 + ['Самоспасатели'] * 6,
        'description': [
            'Горелка газовая ЗСУ-ПИ-38-350-IP65',
            'Горелка газовая ЗСУ-ПИ-38-350-IP66', 
            'Горелка газовая ЗСУ-ПИ-38-350-IP67',
            'Горелка электрическая R93A M.PR.S.RU.A.8.50',
            'Горелка электрическая R93A M.PR.S.RU.A.8.51',
            'Горелка электрическая R93A M.PR.S.RU.A.8.52',
            'Самоспасатель изолирующий СПИ-20 ТР ТС 019/2011',
            'Самоспасатель изолирующий СПИ-20 ТР ТС 019/2011',
            'Самоспасатель изолирующий СПИ-20 ТР ТС 019/2011',
            'Самоспасатель фильтрующий ЗЕВС 30У ТР ТС 019/2011',
            'Самоспасатель фильтрующий ЗЕВС 30У ТР ТС 019/2011',
            'Самоспасатель фильтрующий ЗЕВС 30У ТР ТС 019/2011'
        ],
        'characteristic': [
            'Вид продукции', 'Тип горелки', 'Модель',
            'Вид продукции', 'Тип горелки', 'Модель', 
            'Вид продукции', 'Тип самоспасателя', 'Модель',
            'Вид продукции', 'Тип самоспасателя', 'Модель'
        ],
        'value': [
            'Горелка', 'газовая', 'ЗСУ-ПИ-38-350-IP67',
            'Горелка', 'электрическая', 'R93A M.PR.S.RU.A.8.52',
            'Самоспасатель', 'изолирующий', 'СПИ-20 ТР ТС 019/2011',
            'Самоспасатель', 'фильтрующий', 'ЗЕВС 30У ТР ТС 019/2011'
        ]
    })
    
    parser = NomenclatureParser()
    parser.train(normalized_data)
    
    unnormalized_data = create_sample_unnormalized_data()
    results = parser.process_unnormalized_data(unnormalized_data)
    
    print("Демонстрационные результаты:")
    print(results)
    
    return parser, results

if __name__ == "__main__":
    parser, results = main()

## Основная модель

In [ ]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error, r2_score
from rapidfuzz import process, fuzz
import joblib
from typing import Dict, List, Tuple, Any, Optional, Union
from collections import defaultdict

class NumericCharacteristicExtractor:
    """Класс для извлечения числовых характеристик"""
    
    def __init__(self):
        self.numeric_patterns = {
            'длина': [
                r'(\d+[.,]?\d*)\s*(мм|см|м|метр|сантиметр|миллиметр)',
                r'длина[:\s]*(\d+[.,]?\d*)\s*(мм|см|м)?',
                r'(\d+[.,]?\d*)\s*(мм|см|м)\s*длина'
            ],
            'ширина': [
                r'ширина[:\s]*(\d+[.,]?\d*)\s*(мм|см|м)?',
                r'(\d+[.,]?\d*)\s*(мм|см|м)\s*ширина'
            ],
            'высота': [
                r'высота[:\s]*(\d+[.,]?\d*)\s*(мм|см|м)?',
                r'(\d+[.,]?\d*)\s*(мм|см|м)\s*высота'
            ],
            'диаметр': [
                r'диаметр[:\s]*(\d+[.,]?\d*)\s*(мм|см|м)?',
                r'(\d+[.,]?\d*)\s*(мм|см|м)\s*диаметр',
                r'ø\s*(\d+[.,]?\d*)\s*(мм|см|м)?'
            ],
            'вес': [
                r'вес[:\s]*(\d+[.,]?\d*)\s*(г|кг|т|грамм|килограмм|тонн)?',
                r'(\d+[.,]?\d*)\s*(г|кг|т)\s*вес',
                r'масса[:\s]*(\d+[.,]?\d*)\s*(г|кг|т)?'
            ],
            'объем': [
                r'объем[:\s]*(\d+[.,]?\d*)\s*(мл|л|см3|м3)?',
                r'(\d+[.,]?\d*)\s*(мл|л|см3|м3)\s*объем'
            ],
            'мощность': [
                r'мощность[:\s]*(\d+[.,]?\d*)\s*(вт|квт|ватт|киловатт)?',
                r'(\d+[.,]?\d*)\s*(вт|квт)\s*мощность'
            ],
            'напряжение': [
                r'напряжение[:\s]*(\d+[.,]?\d*)\s*(в|вольт|кв)?',
                r'(\d+[.,]?\d*)\s*(в|вольт)\s*напряжение'
            ],
            'ток': [
                r'ток[:\s]*(\d+[.,]?\d*)\s*(а|ампер|ма)?',
                r'(\d+[.,]?\d*)\s*(а|ампер)\s*ток',
                r'сила тока[:\s]*(\d+[.,]?\d*)\s*(а|ампер)?'
            ],
            'температура': [
                r'температура[:\s]*([+-]?\d+[.,]?\d*)\s*(°c|°f|с|градус)?',
                r'([+-]?\d+[.,]?\d*)\s*(°c|°f)\s*температура',
                r't[:\s]*([+-]?\d+[.,]?\d*)\s*(°c|°f)?'
            ],
            'давление': [
                r'давление[:\s]*(\d+[.,]?\d*)\s*(па|кпа|мпа|бар|атм)?',
                r'(\d+[.,]?\d*)\s*(па|бар|атм)\s*давление'
            ],
            'скорость': [
                r'скорость[:\s]*(\d+[.,]?\d*)\s*(м/с|км/ч|об/мин)?',
                r'(\d+[.,]?\d*)\s*(м/с|км/ч)\s*скорость'
            ],
            'количество': [
                r'(\d+)\s*(шт|штук|единиц)',
                r'количество[:\s]*(\d+)\s*(шт|штук)?',
                r'(\d+)\s*штук'
            ],
            'размер': [
                r'размер[:\s]*(\d+[.,]?\d*)\s*(мм|см|м)?',
                r'(\d+[.,]?\d*)\s*(мм|см|м)\s*размер'
            ]
        }
        
        self.unit_converters = {
            'длина': self._convert_length,
            'вес': self._convert_weight,
            'объем': self._convert_volume,
            'мощность': self._convert_power,
            'напряжение': self._convert_voltage,
            'ток': self._convert_current,
            'температура': self._convert_temperature,
            'давление': self._convert_pressure
        }
    
    def _convert_length(self, value: float, unit: str) -> Tuple[float, str]:
        """Конвертация единиц длины в мм"""
        unit = unit.lower()
        if unit in ['м', 'метр']:
            return value * 1000, 'мм'
        elif unit in ['см', 'сантиметр']:
            return value * 10, 'мм'
        else:
            return value, 'мм'
    
    def _convert_weight(self, value: float, unit: str) -> Tuple[float, str]:
        """Конвертация единиц веса в кг"""
        unit = unit.lower()
        if unit in ['г', 'грамм']:
            return value / 1000, 'кг'
        elif unit in ['т', 'тонн']:
            return value * 1000, 'кг'
        else:
            return value, 'кг'
    
    def _convert_volume(self, value: float, unit: str) -> Tuple[float, str]:
        """Конвертация единиц объема в литры"""
        unit = unit.lower()
        if unit in ['мл']:
            return value / 1000, 'л'
        elif unit in ['м3']:
            return value * 1000, 'л'
        else:
            return value, 'л'
    
    def _convert_power(self, value: float, unit: str) -> Tuple[float, str]:
        """Конвертация единиц мощности в Вт"""
        unit = unit.lower()
        if unit in ['квт', 'киловатт']:
            return value * 1000, 'вт'
        else:
            return value, 'вт'
    
    def _convert_voltage(self, value: float, unit: str) -> Tuple[float, str]:
        """Конвертация единиц напряжения в В"""
        unit = unit.lower()
        if unit in ['кв', 'киловольт']:
            return value * 1000, 'в'
        else:
            return value, 'в'
    
    def _convert_current(self, value: float, unit: str) -> Tuple[float, str]:
        """Конвертация единиц тока в А"""
        unit = unit.lower()
        if unit in ['ма']:
            return value / 1000, 'а'
        else:
            return value, 'а'
    
    def _convert_temperature(self, value: float, unit: str) -> Tuple[float, str]:
        """Конвертация единиц температуры в °C"""
        unit = unit.lower()
        if unit in ['°f']:
            return (value - 32) * 5/9, '°c'
        else:
            return value, '°c'
    
    def _convert_pressure(self, value: float, unit: str) -> Tuple[float, str]:
        """Конвертация единиц давления в бар"""
        unit = unit.lower()
        if unit in ['па']:
            return value / 100000, 'бар'
        elif unit in ['кпа']:
            return value / 100, 'бар'
        elif unit in ['мпа']:
            return value * 10, 'бар'
        elif unit in ['атм']:
            return value * 1.01325, 'бар'
        else:
            return value, 'бар'
    
    def extract_numeric_value(self, char_name: str, text: str) -> Optional[Dict[str, Any]]:
        """Извлечение числового значения для характеристики"""
        if char_name not in self.numeric_patterns:
            return None
        
        patterns = self.numeric_patterns[char_name]
        text_lower = text.lower()
        
        for pattern in patterns:
            matches = re.findall(pattern, text_lower, re.IGNORECASE)
            if matches:
                for match in matches:
                    # match может быть кортежем (значение, единица) или (значение, единица, что-то еще)
                    if len(match) >= 1:
                        value_str = match[0].replace(',', '.')
                        unit = match[1] if len(match) > 1 and match[1] else ''
                        
                        try:
                            # Пробуем преобразовать в число
                            if '.' in value_str:
                                value = float(value_str)
                            else:
                                value = int(value_str)
                            
                            # Конвертируем единицы измерения если нужно
                            if char_name in self.unit_converters and unit:
                                value, standard_unit = self.unit_converters[char_name](value, unit)
                            else:
                                standard_unit = unit
                            
                            return {
                                'value': value,
                                'unit': standard_unit,
                                'original_value': value_str,
                                'original_unit': unit
                            }
                        except (ValueError, TypeError):
                            continue
        
        return None
    
    def extract_all_numeric_values(self, text: str) -> Dict[str, Dict[str, Any]]:
        """Извлечение всех числовых значений из текста"""
        results = {}
        
        for char_name in self.numeric_patterns.keys():
            value_info = self.extract_numeric_value(char_name, text)
            if value_info:
                results[char_name] = value_info
        
        return results

class EnhancedNomenclatureParser:
    """Улучшенный парсер с поддержкой числовых характеристик"""
    
    def __init__(self):
        self.category_classifier = None
        self.vectorizer = None
        self.label_encoder = LabelEncoder()
        self.category_characteristics = defaultdict(list)
        self.characteristic_types = defaultdict(str)  # characteristic -> type (text/numeric)
        self.numeric_extractor = NumericCharacteristicExtractor()
        self.value_patterns = defaultdict(list)
        self.is_trained = False
        
    def load_normalized_data(self, file_path: str) -> pd.DataFrame:
        """Загрузка нормализованных данных"""
        try:
            df = pd.read_csv(file_path, sep='\t', encoding='utf-8')
            if df.shape[1] == 1:
                df = pd.read_csv(file_path, sep=',', encoding='utf-8')
        except:
            df = pd.read_csv(file_path, encoding='utf-8', sep=None, engine='python')
        
        required_columns = ['nomenclature', 'category', 'description', 'characteristic', 'value']
        missing_columns = [col for col in required_columns if col not in df.columns]
        
        if missing_columns:
            raise ValueError(f"Отсутствуют обязательные колонки: {missing_columns}")
        
        print(f"Загружено {len(df)} записей нормализованных данных")
        print(f"Количество категорий: {df['category'].nunique()}")
        print(f"Количество характеристик: {df['characteristic'].nunique()}")
        
        return df
    
    def preprocess_text(self, text: str) -> str:
        """Предобработка текста"""
        if pd.isna(text):
            return ""
        
        text = str(text).lower()
        text = re.sub(r'[^\w\s\d\.\,\-\+\/\±]', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    
    def analyze_characteristic_types(self, normalized_data: pd.DataFrame):
        """Анализ типов характеристик (текстовые/числовые)"""
        
        for category in normalized_data['category'].unique():
            category_data = normalized_data[normalized_data['category'] == category]
            
            for char_name in category_data['characteristic'].unique():
                char_data = category_data[category_data['characteristic'] == char_name]
                values = char_data['value'].dropna()
                
                # Определяем тип характеристики
                is_numeric = False
                numeric_count = 0
                total_count = len(values)
                
                for value in values:
                    value_str = str(value)
                    # Проверяем, является ли значение числовым
                    if re.match(r'^-?\d+[.,]?\d*$', value_str.replace(',', '.')):
                        numeric_count += 1
                    # Проверяем, содержит ли значение число с единицами измерения
                    elif re.search(r'\d+', value_str):
                        # Если есть число, но не только число, считаем смешанным
                        pass
                
                # Если более 50% значений числовые, считаем характеристику числовой
                if total_count > 0 and numeric_count / total_count > 0.5:
                    self.characteristic_types[char_name] = 'numeric'
                else:
                    self.characteristic_types[char_name] = 'text'
                
                print(f"  {char_name}: {self.characteristic_types[char_name]} "
                      f"({numeric_count}/{total_count} числовых)")
    
    def analyze_characteristics(self, normalized_data: pd.DataFrame):
        """Анализ характеристик и их значений"""
        
        # Анализ типов характеристик
        self.analyze_characteristic_types(normalized_data)
        
        # Группируем по категориям и характеристикам
        for category in normalized_data['category'].unique():
            category_data = normalized_data[normalized_data['category'] == category]
            
            for char_name in category_data['characteristic'].unique():
                char_data = category_data[category_data['characteristic'] == char_name]
                
                # Добавляем характеристику в список для категории
                if char_name not in self.category_characteristics[category]:
                    self.category_characteristics[category].append(char_name)
                
                # Для текстовых характеристик анализируем значения
                if self.characteristic_types.get(char_name) == 'text':
                    values = char_data['value'].dropna().unique()
                    
                    for value in values:
                        value_str = str(value).lower()
                        patterns = self._extract_patterns_from_value(value_str)
                        for pattern in patterns:
                            if pattern not in self.value_patterns[char_name]:
                                self.value_patterns[char_name].append(pattern)
        
        print("Анализ характеристик завершен:")
        for category, chars in self.category_characteristics.items():
            numeric_chars = [ch for ch in chars if self.characteristic_types.get(ch) == 'numeric']
            text_chars = [ch for ch in chars if self.characteristic_types.get(ch) == 'text']
            print(f"  {category}: {len(chars)} характеристик "
                  f"({len(numeric_chars)} числовых, {len(text_chars)} текстовых)")
    
    def _extract_patterns_from_value(self, value: str) -> List[str]:
        """Извлечение паттернов из значений характеристик"""
        patterns = []
        
        # Числовые паттерны
        number_patterns = [
            r'\d+[.,]?\d*',  # числа с плавающей точкой
            r'\d+',          # целые числа
        ]
        
        # Текстовые паттерны
        text_patterns = [
            r'[a-zа-я]+',    # слова
            r'[a-zа-я]+\s+[a-zа-я]+',  # словосочетания
        ]
        
        # Специальные паттерны (коды, модели)
        special_patterns = [
            r'[a-zа-я]\d+',           # буква + цифры
            r'\d+[a-zа-я]',           # цифры + буква
            r'[a-zа-я]\d+[a-zа-я]',   # буква + цифры + буква
            r'\d+-\d+',               # числа через дефис
            r'[a-zа-я]+-\d+',         # буквы-числа
        ]
        
        # Проверяем каждый паттерн
        all_patterns = number_patterns + text_patterns + special_patterns
        
        for pattern in all_patterns:
            matches = re.findall(pattern, value, re.IGNORECASE)
            for match in matches:
                if len(match) > 1 and match not in patterns:
                    patterns.append(match)
        
        return patterns
    
    def train_category_classifier(self, normalized_data: pd.DataFrame):
        """Обучение классификатора категорий"""
        
        unique_data = normalized_data.drop_duplicates(subset=['nomenclature', 'category', 'description'])
        
        if len(unique_data) == 0:
            raise ValueError("Недостаточно данных для обучения классификатора")
        
        descriptions = unique_data['description'].apply(self.preprocess_text).tolist()
        categories = unique_data['category'].tolist()
        
        # Кодируем категории
        categories_encoded = self.label_encoder.fit_transform(categories)
        
        # Векторизация текста
        self.vectorizer = TfidfVectorizer(
            max_features=1000,
            ngram_range=(1, 2),
            stop_words=None
        )
        
        X = self.vectorizer.fit_transform(descriptions)
        
        # Обучение модели
        self.category_classifier = RandomForestClassifier(
            n_estimators=100,
            random_state=42,
            class_weight='balanced'
        )
        
        self.category_classifier.fit(X, categories_encoded)
        
        print(f"Классификатор категорий обучен на {len(descriptions)} примерах")
        print(f"Количество категорий: {len(self.label_encoder.classes_)}")
    
    def predict_category(self, description: str) -> str:
        """Предсказание категории для описания"""
        if self.category_classifier is None:
            raise ValueError("Классификатор категорий не обучен")
        
        text = self.preprocess_text(description)
        X = self.vectorizer.transform([text])
        y_pred = self.category_classifier.predict(X)
        return self.label_encoder.inverse_transform(y_pred)[0]
    
    def extract_characteristics(self, category: str, description: str) -> Dict[str, Any]:
        """Извлечение характеристик для указанной категории"""
        if category not in self.category_characteristics:
            return {}
        
        characteristics = {}
        text = self.preprocess_text(description)
        
        for char_name in self.category_characteristics[category]:
            char_type = self.characteristic_types.get(char_name, 'text')
            
            if char_type == 'numeric':
                # Извлечение числовой характеристики
                value_info = self.numeric_extractor.extract_numeric_value(char_name, description)
                if value_info:
                    characteristics[char_name] = value_info
                else:
                    # Если не нашли числовое значение, пробуем текстовое извлечение
                    text_value = self._extract_text_characteristic(char_name, text)
                    if text_value:
                        characteristics[char_name] = {'value': text_value, 'type': 'text'}
            else:
                # Извлечение текстовой характеристики
                text_value = self._extract_text_characteristic(char_name, text)
                if text_value:
                    characteristics[char_name] = {'value': text_value, 'type': 'text'}
        
        return characteristics
    
    def _extract_text_characteristic(self, char_name: str, text: str) -> Optional[str]:
        """Извлечение текстовой характеристики"""
        
        # Специализированные методы для разных характеристик
        specialized_methods = {
            'вид продукции': self._extract_product_type,
            'тип горелки': self._extract_burner_type,
            'тип самоспасателя': self._extract_respirator_type,
            'модель': self._extract_model,
        }
        
        if char_name.lower() in specialized_methods:
            return specialized_methods[char_name.lower()](text)
        else:
            return self._extract_general_text_characteristic(char_name, text)
    
    def _extract_product_type(self, text: str) -> Optional[str]:
        """Извлечение вида продукции"""
        product_types = ['горелка', 'самоспасатель']
        
        for product_type in product_types:
            if product_type in text:
                return product_type.capitalize()
        
        return None
    
    def _extract_burner_type(self, text: str) -> Optional[str]:
        """Извлечение типа горелки"""
        burner_types = ['газовая', 'электрическая']
        
        for burner_type in burner_types:
            if burner_type in text:
                return burner_type
        
        if 'газ' in text:
            return 'газовая'
        elif 'электр' in text:
            return 'электрическая'
        
        return None
    
    def _extract_respirator_type(self, text: str) -> Optional[str]:
        """Извлечение типа самоспасателя"""
        respirator_types = ['изолирующий', 'фильтрующий']
        
        for resp_type in respirator_types:
            if resp_type in text:
                return resp_type
        
        return None
    
    def _extract_model(self, text: str) -> Optional[str]:
        """Извлечение модели/артикула"""
        model_patterns = [
            r'[a-zа-я]{2,}\s*[\-\s]*[a-zа-я]*\s*\d+[\-\s]*\d*[a-zа-я]*',
            r'[a-zа-я]+\s*\d+[a-zа-я]*',
            r'[a-zа-я]+\s*\d+[\s\-]*[a-zа-я]*\s*\d*',
            r'[a-zа-я]+\s*\d+[a-zа-я]+\s*\d*',
        ]
        
        for pattern in model_patterns:
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                best_match = max(matches, key=len)
                return best_match.upper()
        
        return None
    
    def _extract_general_text_characteristic(self, char_name: str, text: str) -> Optional[str]:
        """Общий метод извлечения текстовых характеристик"""
        if char_name in self.value_patterns:
            for pattern in self.value_patterns[char_name]:
                if pattern in text:
                    return pattern
        
        return None
    
    def train(self, normalized_data: pd.DataFrame):
        """Полное обучение модели"""
        print("=== ОБУЧЕНИЕ МОДЕЛИ ===")
        
        # Анализ характеристик
        self.analyze_characteristics(normalized_data)
        
        # Обучение классификатора категорий
        self.train_category_classifier(normalized_data)
        
        self.is_trained = True
        print("Модель успешно обучена")
    
    def process_unnormalized_data(self, unnormalized_data: pd.DataFrame) -> pd.DataFrame:
        """Обработка ненормализованных данных"""
        if not self.is_trained:
            raise ValueError("Модель не обучена. Сначала выполните обучение.")
        
        results = []
        
        for _, row in unnormalized_data.iterrows():
            nomenclature = row.get('nomenclature', '')
            category = row.get('category')
            description = row.get('description', '')
            
            # Если категория не указана, предсказываем её
            if pd.isna(category) or not category:
                try:
                    category = self.predict_category(description)
                    predicted_category = True
                except:
                    category = "unknown"
                    predicted_category = True
            else:
                predicted_category = False
            
            # Извлекаем характеристики
            characteristics = self.extract_characteristics(category, description)
            
            # Создаем результат
            result_row = {
                'nomenclature': nomenclature,
                'category': category,
                'category_predicted': predicted_category,
                'description': description,
                'extracted_characteristics': characteristics
            }
            
            # Добавляем отдельные колонки для каждой характеристики
            for char_name, char_info in characteristics.items():
                if isinstance(char_info, dict) and 'value' in char_info:
                    result_row[char_name] = char_info['value']
                    result_row[f'{char_name}_type'] = char_info.get('type', 'unknown')
                    if 'unit' in char_info:
                        result_row[f'{char_name}_unit'] = char_info['unit']
                else:
                    result_row[char_name] = char_info
            
            results.append(result_row)
        
        return pd.DataFrame(results)
    
    def save_model(self, filepath: str):
        """Сохранение модели"""
        if not self.is_trained:
            raise ValueError("Модель не обучена")
        
        model_data = {
            'category_classifier': self.category_classifier,
            'vectorizer': self.vectorizer,
            'label_encoder': self.label_encoder,
            'category_characteristics': dict(self.category_characteristics),
            'characteristic_types': dict(self.characteristic_types),
            'value_patterns': dict(self.value_patterns),
            'is_trained': self.is_trained
        }
        
        joblib.dump(model_data, filepath)
        print(f"Модель сохранена в {filepath}")
    
    @classmethod
    def load_model(cls, filepath: str) -> 'EnhancedNomenclatureParser':
        """Загрузка модели"""
        model_data = joblib.load(filepath)
        
        parser = cls()
        parser.category_classifier = model_data['category_classifier']
        parser.vectorizer = model_data['vectorizer']
        parser.label_encoder = model_data['label_encoder']
        parser.category_characteristics = defaultdict(list, model_data['category_characteristics'])
        parser.characteristic_types = defaultdict(str, model_data['characteristic_types'])
        parser.value_patterns = defaultdict(list, model_data['value_patterns'])
        parser.is_trained = model_data['is_trained']
        
        print(f"Модель загружена из {filepath}")
        return parser

# Пример использования с числовыми характеристиками
def create_sample_data_with_numeric() -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Создание примера данных с числовыми характеристиками"""
    
    # Нормализованные данные
    normalized_data = pd.DataFrame({
        'nomenclature': ['1267', '1267', '1267', '1267', '1324', '1324', '1324', '1324'],
        'category': ['Горелки теплотехнические'] * 4 + ['Самоспасатели'] * 4,
        'description': [
            'Горелка газовая ЗСУ-ПИ-38-350-IP65',
            'Горелка газовая ЗСУ-ПИ-38-350-IP65', 
            'Горелка газовая ЗСУ-ПИ-38-350-IP65',
            'Горелка газовая ЗСУ-ПИ-38-350-IP65',
            'Самоспасатель изолирующий СПИ-20 ТР ТС 019/2011',
            'Самоспасатель изолирующий СПИ-20 ТР ТС 019/2011',
            'Самоспасатель изолирующий СПИ-20 ТР ТС 019/2011',
            'Самоспасатель изолирующий СПИ-20 ТР ТС 019/2011'
        ],
        'characteristic': [
            'Вид продукции', 'Тип горелки', 'Модель', 'Мощность',
            'Вид продукции', 'Тип самоспасателя', 'Модель', 'Время защиты'
        ],
        'value': [
            'Горелка', 'газовая', 'ЗСУ-ПИ-38-350-IP65', '350',
            'Самоспасатель', 'изолирующий', 'СПИ-20', '20'
        ]
    })
    
    # Ненормализованные данные
    unnormalized_data = pd.DataFrame({
        'nomenclature': ['1234', '5678', '91011'],
        'category': ['Горелки теплотехнические', None, 'Самоспасатели'],
        'description': [
            'Горелка газовая мощностью 350 кВт модель ЗСУ-ПИ-38-350',
            'Электрическая горелка мощность 250 кВт диаметр 150 мм',
            'Самоспасатель изолирующий время защиты 25 минут вес 2.5 кг'
        ]
    })
    
    return normalized_data, unnormalized_data

def main():
    """Основная функция демонстрации"""
    print("=== УЛУЧШЕННАЯ СИСТЕМА ПАРСИНГА С ЧИСЛОВЫМИ ХАРАКТЕРИСТИКАМИ ===\n")
    
    # Инициализация парсера
    parser = EnhancedNomenclatureParser()
    
    try:
        # Загрузка или создание данных
        print("1. ПОДГОТОВКА ДАННЫХ...")
        normalized_data, unnormalized_data = create_sample_data_with_numeric()
        
        print("Нормализованные данные:")
        print(normalized_data)
        print("\nДанные для обработки:")
        print(unnormalized_data)
        
        # Обучение модели
        print("\n2. ОБУЧЕНИЕ МОДЕЛИ...")
        parser.train(normalized_data)
        
        # Обработка данных
        print("\n3. ОБРАБОТКА ДАННЫХ...")
        results = parser.process_unnormalized_data(unnormalized_data)
        
        print("\n4. РЕЗУЛЬТАТЫ ОБРАБОТКИ:")
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 1000)
        print(results)
        
        # Детальный анализ извлеченных характеристик
        print("\n5. ДЕТАЛЬНЫЙ АНАЛИЗ ХАРАКТЕРИСТИК:")
        for _, row in results.iterrows():
            print(f"\nНоменклатура: {row['nomenclature']}")
            print(f"Категория: {row['category']}")
            if 'extracted_characteristics' in row and row['extracted_characteristics']:
                for char_name, char_info in row['extracted_characteristics'].items():
                    print(f"  {char_name}: {char_info}")
        
        # Сохранение модели
        print("\n6. СОХРАНЕНИЕ МОДЕЛИ...")
        parser.save_model('enhanced_nomenclature_parser.pkl')
        
        return parser, results
        
    except Exception as e:
        print(f"Ошибка: {e}")
        import traceback
        traceback.print_exc()



In [5]:
if __name__ == "__main__":
    parser, results = main()

=== УЛУЧШЕННАЯ СИСТЕМА ПАРСИНГА С ЧИСЛОВЫМИ ХАРАКТЕРИСТИКАМИ ===

1. ПОДГОТОВКА ДАННЫХ...
Нормализованные данные:
  nomenclature                  category  \
0         1267  Горелки теплотехнические   
1         1267  Горелки теплотехнические   
2         1267  Горелки теплотехнические   
3         1267  Горелки теплотехнические   
4         1324             Самоспасатели   
5         1324             Самоспасатели   
6         1324             Самоспасатели   
7         1324             Самоспасатели   

                                       description     characteristic  \
0               Горелка газовая ЗСУ-ПИ-38-350-IP65      Вид продукции   
1               Горелка газовая ЗСУ-ПИ-38-350-IP65        Тип горелки   
2               Горелка газовая ЗСУ-ПИ-38-350-IP65             Модель   
3               Горелка газовая ЗСУ-ПИ-38-350-IP65           Мощность   
4  Самоспасатель изолирующий СПИ-20 ТР ТС 019/2011      Вид продукции   
5  Самоспасатель изолирующий СПИ-20 ТР ТС 019/2011 

## Отчет

In [18]:
# utils.py
import pandas as pd
import numpy as np
import json
from typing import Dict, List, Any

def analyze_processing_results(processed_data: pd.DataFrame, parser=None):
    """Расширенный анализ результатов обработки с учетом числовых характеристик"""
    print("=== РАСШИРЕННЫЙ АНАЛИЗ РЕЗУЛЬТАТОВ ===")
    
    # Базовая статистика
    print(f"\n1. ОБЩАЯ СТАТИСТИКА:")
    print(f"   Всего обработано записей: {len(processed_data)}")
    print(f"   Уникальных номенклатур: {processed_data['nomenclature'].nunique()}")
    print(f"   Уникальных категорий: {processed_data['category'].nunique()}")
    
    # Статистика по категориям
    print(f"\n2. РАСПРЕДЕЛЕНИЕ ПО КАТЕГОРИЯМ:")
    category_stats = processed_data['category'].value_counts()
    for category, count in category_stats.items():
        print(f"   {category}: {count} записей ({count/len(processed_data)*100:.1f}%)")
    
    # Статистика по предсказанным категориям
    if 'category_predicted' in processed_data.columns:
        predicted_stats = processed_data['category_predicted'].value_counts()
        print(f"\n3. СТАТИСТИКА ПРЕДСКАЗАНИЯ КАТЕГОРИЙ:")
        print(f"   Предсказано категорий: {predicted_stats.get(True, 0)}")
        print(f"   Известно категорий: {predicted_stats.get(False, 0)}")
    
    # Анализ характеристик
    print(f"\n4. АНАЛИЗ ИЗВЛЕЧЕННЫХ ХАРАКТЕРИСТИК:")
    
    # Находим колонки с характеристиками (исключаем служебные)
    service_columns = ['nomenclature', 'category', 'description', 'category_predicted', 'extracted_characteristics']
    characteristic_cols = [col for col in processed_data.columns if col not in service_columns]
    
    # Разделяем на основные характеристики и дополнительные поля (unit, type)
    main_characteristics = []
    additional_fields = []
    
    for col in characteristic_cols:
        if any(col.endswith(suffix) for suffix in ['_type', '_unit', '_original']):
            additional_fields.append(col)
        else:
            main_characteristics.append(col)
    
    print(f"   Всего извлечено характеристик: {len(main_characteristics)}")
    
    # Анализ каждой характеристики
    char_analysis = []
    for char in main_characteristics:
        char_data = processed_data[char]
        fill_rate = char_data.notna().mean() * 100
        unique_values = char_data.nunique()
        
        # Определяем тип характеристики
        char_type = "неизвестно"
        type_col = f"{char}_type"
        if type_col in processed_data.columns:
            type_values = processed_data[type_col].dropna().unique()
            if len(type_values) > 0:
                char_type = type_values[0]
        
        # Анализ единиц измерения для числовых характеристик
        unit_info = ""
        unit_col = f"{char}_unit"
        if unit_col in processed_data.columns:
            units = processed_data[unit_col].dropna().unique()
            if len(units) > 0:
                unit_info = f", единицы: {', '.join(map(str, units))}"
        
        char_analysis.append({
            'characteristic': char,
            'fill_rate': fill_rate,
            'unique_values': unique_values,
            'type': char_type,
            'unit_info': unit_info
        })
        
        print(f"   ├─ {char}:")
        print(f"   │  Заполнено: {fill_rate:.1f}%")
        print(f"   │  Уникальных значений: {unique_values}")
        print(f"   │  Тип: {char_type}{unit_info}")
    
    # Анализ числовых характеристик
    numeric_chars = [char for char in char_analysis if char['type'] == 'numeric']
    if numeric_chars:
        print(f"\n5. АНАЛИЗ ЧИСЛОВЫХ ХАРАКТЕРИСТИК:")
        for char_info in numeric_chars:
            char_name = char_info['characteristic']
            numeric_data = processed_data[char_name].dropna()
            
            if len(numeric_data) > 0:
                # Преобразуем в числа, если возможно
                try:
                    numeric_series = pd.to_numeric(numeric_data, errors='coerce')
                    numeric_series = numeric_series.dropna()
                    
                    if len(numeric_series) > 0:
                        print(f"   ├─ {char_name}:")
                        print(f"   │  Минимум: {numeric_series.min():.2f}")
                        print(f"   │  Максимум: {numeric_series.max():.2f}")
                        print(f"   │  Среднее: {numeric_series.mean():.2f}")
                        print(f"   │  Медиана: {numeric_series.median():.2f}")
                        print(f"   │  Стандартное отклонение: {numeric_series.std():.2f}")
                except:
                    print(f"   ├─ {char_name}: не удалось проанализировать числовые значения")
    
    # Анализ качества извлечения по категориям
    if parser and hasattr(parser, 'category_characteristics'):
        print(f"\n6. АНАЛИЗ ПОЛНОТЫ ИЗВЛЕЧЕНИЯ ПО КАТЕГОРИЯМ:")
        for category in processed_data['category'].unique():
            category_data = processed_data[processed_data['category'] == category]
            expected_chars = parser.category_characteristics.get(category, [])
            
            if expected_chars:
                extracted_count = 0
                for char in expected_chars:
                    if char in processed_data.columns:
                        fill_rate = category_data[char].notna().mean() * 100
                        extracted_count += 1 if fill_rate > 0 else 0
                
                completeness = extracted_count / len(expected_chars) * 100
                print(f"   ├─ {category}:")
                print(f"   │  Ожидаемые характеристики: {len(expected_chars)}")
                print(f"   │  Извлеченные характеристики: {extracted_count}")
                print(f"   │  Полнота извлечения: {completeness:.1f}%")
    
    # Анализ сложных случаев
    print(f"\n7. АНАЛИЗ СЛОЖНЫХ СЛУЧАЕВ:")
    
    # Записи без извлеченных характеристик
    empty_records = processed_data[processed_data[main_characteristics].isna().all(axis=1)]
    if len(empty_records) > 0:
        print(f"   Записей без извлеченных характеристик: {len(empty_records)}")
        for _, record in empty_records.head(3).iterrows():
            print(f"     - {record['nomenclature']}: {record['description'][:50]}...")
    
    # Записи с частично извлеченными характеристиками
    if parser and hasattr(parser, 'category_characteristics'):
        partial_records = []
        for _, record in processed_data.iterrows():
            category = record['category']
            expected_chars = parser.category_characteristics.get(category, [])
            if expected_chars:
                extracted_chars = sum(1 for char in expected_chars 
                                   if char in record and pd.notna(record.get(char)))
                if 0 < extracted_chars < len(expected_chars):
                    partial_records.append((record, extracted_chars, len(expected_chars)))
        
        if partial_records:
            print(f"   Записей с частичным извлечением: {len(partial_records)}")
            for record, extracted, total in partial_records[:3]:
                print(f"     - {record['nomenclature']}: {extracted}/{total} характеристик")
    
    return {
        'total_records': len(processed_data),
        'categories_count': processed_data['category'].nunique(),
        'characteristics_count': len(main_characteristics),
        'characteristics_analysis': char_analysis,
        'numeric_characteristics_count': len(numeric_chars),
        'empty_records_count': len(empty_records) if 'empty_records' in locals() else 0
    }

def export_results_to_excel(processed_data: pd.DataFrame, filename: str, parser=None):
    """Расширенный экспорт результатов в Excel с учетом числовых характеристик"""
    
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        # Основные результаты
        processed_data.to_excel(writer, sheet_name='Результаты', index=False)
        
        # Статистика характеристик
        service_columns = ['nomenclature', 'category', 'description', 'category_predicted', 'extracted_characteristics']
        characteristic_cols = [col for col in processed_data.columns if col not in service_columns]
        
        main_characteristics = [col for col in characteristic_cols 
                              if not any(col.endswith(suffix) for suffix in ['_type', '_unit', '_original'])]
        
        stats_data = []
        for char in main_characteristics:
            char_data = processed_data[char]
            fill_rate = char_data.notna().mean() * 100
            unique_values = char_data.nunique()
            
            # Тип характеристики
            char_type = "неизвестно"
            type_col = f"{char}_type"
            if type_col in processed_data.columns:
                type_values = processed_data[type_col].dropna().unique()
                if len(type_values) > 0:
                    char_type = type_values[0]
            
            # Единицы измерения
            units = ""
            unit_col = f"{char}_unit"
            if unit_col in processed_data.columns:
                unit_values = processed_data[unit_col].dropna().unique()
                if len(unit_values) > 0:
                    units = ', '.join(map(str, unit_values))
            
            stats_data.append({
                'Характеристика': char,
                'Тип': char_type,
                'Заполнено (%)': fill_rate,
                'Уникальных значений': unique_values,
                'Единицы измерения': units
            })
        
        stats_df = pd.DataFrame(stats_data)
        stats_df.to_excel(writer, sheet_name='Статистика характеристик', index=False)
        
        # Распределение по категориям
        category_stats = processed_data['category'].value_counts().reset_index()
        category_stats.columns = ['Категория', 'Количество']
        category_stats.to_excel(writer, sheet_name='Распределение по категориям', index=False)
        
        # Детальный анализ числовых характеристик
        numeric_stats_data = []
        for char in main_characteristics:
            type_col = f"{char}_type"
            if type_col in processed_data.columns and processed_data[type_col].notna().any():
                if 'numeric' in processed_data[type_col].values:
                    numeric_data = processed_data[char].dropna()
                    if len(numeric_data) > 0:
                        try:
                            numeric_series = pd.to_numeric(numeric_data, errors='coerce').dropna()
                            if len(numeric_series) > 0:
                                numeric_stats_data.append({
                                    'Характеристика': char,
                                    'Минимум': numeric_series.min(),
                                    'Максимум': numeric_series.max(),
                                    'Среднее': numeric_series.mean(),
                                    'Медиана': numeric_series.median(),
                                    'Стандартное отклонение': numeric_series.std(),
                                    'Количество значений': len(numeric_series)
                                })
                        except:
                            pass
        
        if numeric_stats_data:
            numeric_stats_df = pd.DataFrame(numeric_stats_data)
            numeric_stats_df.to_excel(writer, sheet_name='Числовые характеристики', index=False)
        
        # Анализ полноты извлечения по категориям
        if parser and hasattr(parser, 'category_characteristics'):
            completeness_data = []
            for category in processed_data['category'].unique():
                category_data = processed_data[processed_data['category'] == category]
                expected_chars = parser.category_characteristics.get(category, [])
                
                if expected_chars:
                    category_stats = {'Категория': category}
                    for char in expected_chars:
                        if char in processed_data.columns:
                            fill_rate = category_data[char].notna().mean() * 100
                            category_stats[char] = f"{fill_rate:.1f}%"
                        else:
                            category_stats[char] = "0%"
                    
                    completeness_data.append(category_stats)
            
            if completeness_data:
                completeness_df = pd.DataFrame(completeness_data)
                completeness_df.to_excel(writer, sheet_name='Полнота извлечения', index=False)
    
    print(f"Результаты экспортированы в {filename}")

def create_detailed_report(processed_data: pd.DataFrame, parser=None, filename: str = "детальный_отчет.html"):
    """Создание детального HTML отчета"""
    
    analysis_results = analyze_processing_results(processed_data, parser)
    
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <title>Детальный отчет по парсингу характеристик</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 20px; }}
            .section {{ margin-bottom: 30px; }}
            .metric {{ margin: 10px 0; }}
            .characteristic {{ margin: 5px 0; padding: 5px; background: #f5f5f5; }}
            .numeric {{ border-left: 4px solid #4CAF50; }}
            .text {{ border-left: 4px solid #2196F3; }}
            table {{ border-collapse: collapse; width: 100%; }}
            th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
            th {{ background-color: #f2f2f2; }}
        </style>
    </head>
    <body>
        <h1>Детальный отчет по парсингу характеристик</h1>
        
        <div class="section">
            <h2>Общая статистика</h2>
            <div class="metric">Всего обработано записей: {analysis_results['total_records']}</div>
            <div class="metric">Уникальных категорий: {analysis_results['categories_count']}</div>
            <div class="metric">Извлеченных характеристик: {analysis_results['characteristics_count']}</div>
            <div class="metric">Числовых характеристик: {analysis_results['numeric_characteristics_count']}</div>
        </div>
        
        <div class="section">
            <h2>Анализ характеристик</h2>
    """
    
    for char_info in analysis_results['characteristics_analysis']:
        type_class = 'numeric' if char_info['type'] == 'numeric' else 'text'
        html_content += f"""
            <div class="characteristic {type_class}">
                <strong>{char_info['characteristic']}</strong><br>
                Тип: {char_info['type']} | Заполнено: {char_info['fill_rate']:.1f}% | 
                Уникальных значений: {char_info['unique_values']}
                {char_info['unit_info']}
            </div>
        """
    
    html_content += """
        </div>
        
        <div class="section">
            <h2>Примеры данных</h2>
            <table>
                <tr>
                    <th>Номенклатура</th>
                    <th>Категория</th>
                    <th>Описание</th>
                    <th>Извлеченные характеристики</th>
                </tr>
    """
    
    # Добавляем несколько примеров
    for _, row in processed_data.head(5).iterrows():
        characteristics = []
        for col in processed_data.columns:
            if col not in ['nomenclature', 'category', 'description', 'category_predicted', 'extracted_characteristics']:
                if pd.notna(row[col]):
                    characteristics.append(f"{col}: {row[col]}")
        
        html_content += f"""
                <tr>
                    <td>{row['nomenclature']}</td>
                    <td>{row['category']}</td>
                    <td>{row['description'][:100]}...</td>
                    <td>{'<br>'.join(characteristics)}</td>
                </tr>
        """
    
    html_content += """
            </table>
        </div>
    </body>
    </html>
    """
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print(f"Детальный отчет создан: {filename}")

def validate_parsing_quality(processed_data: pd.DataFrame, test_data_with_truth: pd.DataFrame):
    """Валидация качества парсинга на тестовых данных с известными значениями"""
    print("=== ВАЛИДАЦИЯ КАЧЕСТВА ПАРСИНГА ===")
    
    if test_data_with_truth.empty:
        print("Нет тестовых данных для валидации")
        return
    
    # Проверяем совпадение категорий
    if 'true_category' in test_data_with_truth.columns and 'category' in processed_data.columns:
        correct_categories = 0
        total_categories = 0
        
        for _, test_row in test_data_with_truth.iterrows():
            nomenclature = test_row['nomenclature']
            true_category = test_row['true_category']
            
            # Ищем соответствующую запись в обработанных данных
            processed_row = processed_data[processed_data['nomenclature'] == nomenclature]
            if not processed_row.empty:
                predicted_category = processed_row.iloc[0]['category']
                if predicted_category == true_category:
                    correct_categories += 1
                total_categories += 1
        
        if total_categories > 0:
            accuracy = correct_categories / total_categories * 100
            print(f"Точность определения категории: {accuracy:.1f}% ({correct_categories}/{total_categories})")
    
    # Проверяем извлечение характеристик
    print("\nКачество извлечения характеристик:")
    characteristic_cols = [col for col in processed_data.columns 
                         if col not in ['nomenclature', 'category', 'description', 'category_predicted', 'extracted_characteristics']]
    
    for char_col in characteristic_cols:
        if char_col in test_data_with_truth.columns:
            correct_values = 0
            total_values = 0
            
            for _, test_row in test_data_with_truth.iterrows():
                nomenclature = test_row['nomenclature']
                true_value = test_row[char_col]
                
                if pd.notna(true_value):
                    processed_row = processed_data[processed_data['nomenclature'] == nomenclature]
                    if not processed_row.empty and pd.notna(processed_row.iloc[0].get(char_col)):
                        predicted_value = processed_row.iloc[0][char_col]
                        # Простое сравнение значений
                        if str(predicted_value) == str(true_value):
                            correct_values += 1
                        total_values += 1
            
            if total_values > 0:
                accuracy = correct_values / total_values * 100
                print(f"  {char_col}: {accuracy:.1f}% ({correct_values}/{total_values})")



## Запуск отчета

In [27]:
# Основной скрипт (добавьте в конец)
if __name__ == "__main__":
    # ... ваш существующий код ...
    
    parser, results = main()
    
    # Используем утилиты для анализа
    print("\n" + "="*60)
    print("АНАЛИЗ РЕЗУЛЬТАТОВ С ПОМОЩЬЮ УТИЛИТ")
    print("="*60)
    
    # Анализ результатов
    analysis_results = analyze_processing_results(results, parser)
    
    # Экспорт в Excel
    export_results_to_excel(results, 'результаты_парсинга.xlsx', parser)
    
    # Создание HTML отчета
    create_detailed_report(results, parser, 'детальный_отчет.html')
    
    print("\nАнализ завершен!")

=== УЛУЧШЕННАЯ СИСТЕМА ПАРСИНГА С ЧИСЛОВЫМИ ХАРАКТЕРИСТИКАМИ ===

1. ПОДГОТОВКА ДАННЫХ...
Нормализованные данные:
  nomenclature                  category                                      description     characteristic               value
0         1267  Горелки теплотехнические               Горелка газовая ЗСУ-ПИ-38-350-IP65      Вид продукции             Горелка
1         1267  Горелки теплотехнические               Горелка газовая ЗСУ-ПИ-38-350-IP65        Тип горелки             газовая
2         1267  Горелки теплотехнические               Горелка газовая ЗСУ-ПИ-38-350-IP65             Модель  ЗСУ-ПИ-38-350-IP65
3         1267  Горелки теплотехнические               Горелка газовая ЗСУ-ПИ-38-350-IP65           Мощность                 350
4         1324             Самоспасатели  Самоспасатель изолирующий СПИ-20 ТР ТС 019/2011      Вид продукции       Самоспасатель
5         1324             Самоспасатели  Самоспасатель изолирующий СПИ-20 ТР ТС 019/2011  Тип самоспасателя    

## Обновление данных

Краткий план действий:
Автоматическое обновление - просто переобучить на новых данных

Ручная настройка - добавить специализированные методы для сложных случаев

Конфигурационная система - для максимальной гибкости

Тестирование - проверить качество на новых классах

Мониторинг - отслеживать производительность

Система спроектирована так, чтобы минимизировать ручную работу при добавлении новых классов!

При добавлении новых классов с новыми характеристиками потребуется несколько шагов для адаптации системы. Вот полный план действий:

1. Автоматическое обновление (минимальные изменения)
Система уже частично поддерживает автоматическое обновление:

In [ ]:
# Если новые данные в том же формате - просто переобучить модель
def update_model_with_new_data(parser, new_normalized_data):
    """Автоматическое обновление модели с новыми данными"""
    print("=== ОБНОВЛЕНИЕ МОДЕЛИ С НОВЫМИ ДАННЫМИ ===")
    
    # Просто переобучаем модель на объединенных данных
    parser.train(new_normalized_data)
    
    print("Модель успешно обновлена!")
    print(f"Новые классы: {list(parser.category_characteristics.keys())}")
    
    return parser

2. Ручное обновление для сложных случаев
Если новые классы требуют специальных правил извлечения:

In [21]:
class ExtendedNomenclatureParser(EnhancedNomenclatureParser):
    """Расширенный парсер с поддержкой новых классов"""
    
    def __init__(self):
        super().__init__()
        # Добавляем специализированные методы для новых классов
        self.specialized_methods.update({
            'трубы': self._extract_pipe_characteristics,
            'кабели': self._extract_cable_characteristics,
            'арматура': self._extract_fittings_characteristics,
        })
        
        # Добавляем паттерны для новых характеристик
        self.numeric_extractor.numeric_patterns.update({
            'толщина стенки': [
                r'толщина стенки[:\s]*(\d+[.,]?\d*)\s*(мм|см)?',
                r'стенка[:\s]*(\d+[.,]?\d*)\s*(мм|см)?',
            ],
            'сечение': [
                r'сечение[:\s]*(\d+[.,]?\d*)\s*(мм2|мм²|мм\^2)?',
                r'(\d+[.,]?\d*)\s*(мм2|мм²)\s*сечение',
            ],
            'диаметр условный': [
                r'ду[:\s]*(\d+[.,]?\d*)\s*(мм)?',
                r'условный диаметр[:\s]*(\d+[.,]?\d*)\s*(мм)?',
            ]
        })
    
    def _extract_pipe_characteristics(self, text: str) -> Dict[str, Any]:
        """Извлечение характеристик для труб"""
        characteristics = {}
        text_lower = text.lower()
        
        # Материал трубы
        materials = ['пнд', 'пвх', 'пп', 'сталь', 'чугун', 'медь']
        for material in materials:
            if material in text_lower:
                characteristics['материал'] = material.upper()
                break
        
        # Тип трубы
        if 'напорная' in text_lower:
            characteristics['тип трубы'] = 'напорная'
        elif 'безнапорная' in text_lower:
            characteristics['тип трубы'] = 'безнапорная'
        
        return characteristics
    
    def _extract_cable_characteristics(self, text: str) -> Dict[str, Any]:
        """Извлечение характеристик для кабелей"""
        characteristics = {}
        text_lower = text.lower()
        
        # Марка кабеля
        cable_brands = ['ввг', 'ввгнг', 'пвс', 'шввп', 'кабель', 'провод']
        for brand in cable_brands:
            if brand in text_lower:
                characteristics['марка'] = brand.upper()
                break
        
        # Количество жил
        cores_match = re.search(r'(\d+)\s*[хx]\s*(\d+)', text_lower)
        if cores_match:
            characteristics['жилы'] = f"{cores_match.group(1)}x{cores_match.group(2)}"
        
        return characteristics
    
    def _extract_fittings_characteristics(self, text: str) -> Dict[str, Any]:
        """Извлечение характеристик для арматуры"""
        characteristics = {}
        text_lower = text.lower()
        
        # Тип соединения
        connections = ['резьба', 'фланец', 'сварка', 'муфта']
        for connection in connections:
            if connection in text_lower:
                characteristics['тип соединения'] = connection
                break
        
        return characteristics
    
    def _extract_single_characteristic(self, char_name: str, text: str) -> Optional[str]:
        """Переопределяем метод для поддержки новых классов"""
        # Сначала пробуем специализированные методы
        category = self._infer_category_from_text(text)
        if category in self.specialized_methods:
            specialized_chars = self.specialized_methods[category](text)
            if char_name in specialized_chars:
                return specialized_chars[char_name]
        
        # Затем стандартные методы
        return super()._extract_single_characteristic(char_name, text)
    
    def _infer_category_from_text(self, text: str) -> str:
        """Определяем категорию по тексту для специализированных методов"""
        text_lower = text.lower()
        
        if any(word in text_lower for word in ['труба', 'трубка', 'трубопровод']):
            return 'трубы'
        elif any(word in text_lower for word in ['кабель', 'провод', 'кабельный']):
            return 'кабели'
        elif any(word in text_lower for word in ['арматура', 'фитинг', 'муфта', 'отвод']):
            return 'арматура'
        else:
            return 'общий'

## добавление с YAML

In [ ]:
# config.py
import yaml

CLASS_CONFIG = """
# Конфигурация классов и их характеристик
классы:
  горелки теплотехнические:
    тип: оборудование
    характеристики:
      вид продукции:
        тип: текст
        паттерны: ["горелка", "burner"]
        обязательная: true
      
      тип горелки:
        тип: текст  
        паттерны: ["газовая", "электрическая"]
        значения: ["газовая", "электрическая"]
      
      мощность:
        тип: числовая
        единицы: ["квт", "вт"]
        паттерны: ["мощность", "power"]
      
      модель:
        тип: текст
        паттерны: ["модель", "model", "артикул"]

  трубы:
    тип: материалы
    характеристики:
      материал:
        тип: текст
        паттерны: ["материал", "material"]
        значения: ["ПНД", "ПВХ", "ПП", "сталь"]
      
      диаметр:
        тип: числовая
        единицы: ["мм", "дюйм"]
        паттерны: ["диаметр", "ø", "dn"]
      
      толщина стенки:
        тип: числовая
        единицы: ["мм"]
        паттерны: ["толщина стенки", "стенка"]
      
      длина:
        тип: числовая
        единицы: ["м", "мм"]
        паттерны: ["длина", "length"]

  кабели:
    тип: материалы  
    характеристики:
      марка:
        тип: текст
        паттерны: ["марка", "brand"]
        значения: ["ВВГ", "ВВГнг", "ПВС", "ШВВП"]
      
      сечение:
        тип: числовая
        единицы: ["мм2", "мм²"]
        паттерны: ["сечение", "площадь"]
      
      количество жил:
        тип: числовая
        паттерны: ["жилы", "cores", "количество жил"]
      
      напряжение:
        тип: числовая
        единицы: ["в", "вольт"]
        паттерны: ["напряжение", "voltage"]
"""


## Установка зависимостей

In [29]:
!pip install pyyaml pandas scikit-learn rapidfuzz joblib

DEPRECATION: desc 2.1.1 has a non-standard dependency specifier matplotlib>=2.2pydot. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of desc or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Создание конфигурируемого парсера

In [30]:
# configurable_parser.py
import yaml
import os
import pandas as pd
import re
from typing import Dict, List, Any, Optional
#from enhanced_nomenclature_parser import EnhancedNomenclatureParser

class ConfigurableNomenclatureParser(EnhancedNomenclatureParser):
    """
    Универсальный парсер с конфигурацией из YAML-файла
    """
    
    def __init__(self, config_path: str = "config.yaml"):
        super().__init__()
        self.config_path = config_path
        self.config = self._load_config()
        self._apply_config()
        print("Конфигурируемый парсер инициализирован")
    
    def _load_config(self) -> Dict[str, Any]:
        """Загрузка конфигурации из YAML-файла"""
        if not os.path.exists(self.config_path):
            raise FileNotFoundError(f"Конфигурационный файл не найден: {self.config_path}")
        
        with open(self.config_path, 'r', encoding='utf-8') as f:
            config = yaml.safe_load(f)
        
        print(f"Загружена конфигурация для {len(config.get('классы', {}))} классов")
        return config
    
    def _apply_config(self):
        """Применение конфигурации к парсеру"""
        if 'классы' not in self.config:
            print("В конфигурации не найдены классы")
            return
        
        # Очищаем существующие характеристики
        self.category_characteristics.clear()
        self.characteristic_types.clear()
        
        # Загружаем классы и их характеристики из конфигурации
        for class_name, class_config in self.config['классы'].items():
            if 'характеристики' in class_config:
                characteristics = list(class_config['характеристики'].keys())
                self.category_characteristics[class_name] = characteristics
                
                # Устанавливаем типы характеристик
                for char_name, char_config in class_config['характеристики'].items():
                    self.characteristic_types[char_name] = char_config.get('тип', 'текст')
        
        # Обновляем паттерны для числового экстрактора
        self._update_numeric_patterns()
        
        print("Конфигурация применена успешно")
    
    def _update_numeric_patterns(self):
        """Обновление паттернов для числовых характеристик"""
        for class_name, class_config in self.config['классы'].items():
            if 'характеристики' in class_config:
                for char_name, char_config in class_config['характеристики'].items():
                    if char_config.get('тип') == 'числовая':
                        patterns = self._create_numeric_patterns(char_name, char_config)
                        self.numeric_extractor.numeric_patterns[char_name] = patterns
    
    def _create_numeric_patterns(self, char_name: str, char_config: Dict) -> List[str]:
        """Создание паттернов для числовой характеристики"""
        patterns = []
        units = "|".join(char_config.get('единицы', ['']))
        
        for pattern in char_config.get('паттерны', []):
            # Паттерн: характеристика: число единица
            patterns.append(f"{pattern}[\\s:]*([\\d.,]+)\\s*({units})?")
            # Паттерн: число единица характеристика
            patterns.append(f"([\\d.,]+)\\s*({units})?\\s*{pattern}")
        
        return patterns
    
    def add_new_class(self, class_name: str, characteristics_config: Dict):
        """Динамическое добавление нового класса"""
        if class_name not in self.config['классы']:
            self.config['классы'][class_name] = characteristics_config
            self._apply_config()
            print(f"Добавлен новый класс: {class_name}")
        else:
            print(f"Класс {class_name} уже существует")
    
    def update_class(self, class_name: str, characteristics_config: Dict):
        """Обновление существующего класса"""
        if class_name in self.config['классы']:
            self.config['классы'][class_name] = characteristics_config
            self._apply_config()
            print(f"Класс {class_name} обновлен")
        else:
            print(f"Класс {class_name} не найден")
    
    def save_config(self, config_path: str = None):
        """Сохранение текущей конфигурации в файл"""
        save_path = config_path or self.config_path
        
        with open(save_path, 'w', encoding='utf-8') as f:
            yaml.dump(self.config, f, allow_unicode=True, default_flow_style=False)
        
        print(f"Конфигурация сохранена в {save_path}")
    
    def get_class_info(self, class_name: str) -> Dict:
        """Получение информации о классе"""
        return self.config['классы'].get(class_name, {})
    
    def list_classes(self) -> List[str]:
        """Список всех классов"""
        return list(self.config['классы'].keys())
    
    def get_characteristics_for_class(self, class_name: str) -> List[str]:
        """Получение характеристик для класса"""
        class_config = self.config['классы'].get(class_name, {})
        return list(class_config.get('характеристики', {}).keys())

## Использование конфигурируемого парсера

In [31]:
# main.py
#from configurable_parser import ConfigurableNomenclatureParser
import pandas as pd

def main():
    # 1. Инициализация парсера с конфигурацией
    print("=== ИНИЦИАЛИЗАЦИЯ КОНФИГУРИРУЕМОГО ПАРСЕРА ===")
    parser = ConfigurableNomenclatureParser("config.yaml")
    
    # 2. Просмотр доступных классов
    print("\nДоступные классы:")
    for class_name in parser.list_classes():
        characteristics = parser.get_characteristics_for_class(class_name)
        print(f"  {class_name}: {len(characteristics)} характеристик")
    
    # 3. Загрузка нормализованных данных для обучения
    print("\n=== ЗАГРУЗКА ДАННЫХ ДЛЯ ОБУЧЕНИЯ ===")
    try:
        normalized_data = parser.load_normalized_data("normalized.txt")
        
        # 4. Обучение модели
        print("\n=== ОБУЧЕНИЕ МОДЕЛИ ===")
        parser.train(normalized_data)
        
    except FileNotFoundError:
        print("Файл с нормализованными данными не найден, создаем демо-данные...")
        normalized_data = create_demo_data()
        parser.train(normalized_data)
    
    # 5. Обработка тестовых данных
    print("\n=== ОБРАБОТКА ТЕСТОВЫХ ДАННЫХ ===")
    test_data = pd.DataFrame({
        'nomenclature': ['001', '002', '003', '004'],
        'category': ['трубы', None, 'кабели', 'горелки теплотехнические'],
        'description': [
            'Труба ПНД диаметр 32 мм толщина стенки 3 мм длина 6 м',
            'Кабель ВВГнг 3х2.5 сечение 2.5 мм² напряжение 660В',
            'Провод ПВС 2х1.5 длина 50 м',
            'Горелка газовая модель ZSU-PI-38 мощность 350 кВт'
        ]
    })
    
    results = parser.process_unnormalized_data(test_data)
    
    print("\nРезультаты обработки:")
    for _, row in results.iterrows():
        print(f"\nНоменклатура: {row['nomenclature']}")
        print(f"Категория: {row['category']}")
        print(f"Описание: {row['description']}")
        
        # Выводим извлеченные характеристики
        for col in results.columns:
            if col not in ['nomenclature', 'category', 'description', 'category_predicted', 'extracted_characteristics']:
                if pd.notna(row[col]):
                    print(f"  {col}: {row[col]}")
    
    # 6. Сохранение модели
    print("\n=== СОХРАНЕНИЕ МОДЕЛИ ===")
    parser.save_model('configurable_parser.pkl')
    
    return parser, results

def create_demo_data():
    """Создание демонстрационных данных"""
    return pd.DataFrame({
        'nomenclature': ['1267', '1267', '1267', '2356', '2356', '2356'],
        'category': ['горелки теплотехнические', 'горелки теплотехнические', 'горелки теплотехнические', 
                    'самоспасатели', 'самоспасатели', 'самоспасатели'],
        'description': [
            'Горелка газовая ЗСУ-ПИ-38-350-IP65',
            'Горелка газовая ЗСУ-ПИ-38-350-IP65',
            'Горелка газовая ЗСУ-ПИ-38-350-IP65',
            'Самоспасатель изолирующий СПИ-20 ТР ТС 019/2011',
            'Самоспасатель изолирующий СПИ-20 ТР ТС 019/2011',
            'Самоспасатель изолирующий СПИ-20 ТР ТС 019/2011'
        ],
        'characteristic': ['вид продукции', 'тип горелки', 'модель', 
                          'вид продукции', 'тип самоспасателя', 'модель'],
        'value': ['Горелка', 'газовая', 'ЗСУ-ПИ-38-350-IP65',
                 'Самоспасатель', 'изолирующий', 'СПИ-20 ТР ТС 019/2011']
    })

if __name__ == "__main__":
    parser, results = main()

=== ИНИЦИАЛИЗАЦИЯ КОНФИГУРИРУЕМОГО ПАРСЕРА ===
Загружена конфигурация для 4 классов
Конфигурация применена успешно
Конфигурируемый парсер инициализирован

Доступные классы:
  горелки теплотехнические: 4 характеристик
  самоспасатели: 4 характеристик
  трубы: 4 характеристик
  кабели: 4 характеристик

=== ЗАГРУЗКА ДАННЫХ ДЛЯ ОБУЧЕНИЯ ===
Файл с нормализованными данными не найден, создаем демо-данные...
=== ОБУЧЕНИЕ МОДЕЛИ ===
  вид продукции: text (0/1 числовых)
  тип горелки: text (0/1 числовых)
  модель: text (0/1 числовых)
  вид продукции: text (0/1 числовых)
  тип самоспасателя: text (0/1 числовых)
  модель: text (0/1 числовых)
Анализ характеристик завершен:
  горелки теплотехнические: 4 характеристик (0 числовых, 3 текстовых)
  самоспасатели: 4 характеристик (0 числовых, 3 текстовых)
  трубы: 4 характеристик (0 числовых, 0 текстовых)
  кабели: 4 характеристик (0 числовых, 0 текстовых)
Классификатор категорий обучен на 2 примерах
Количество категорий: 2
Модель успешно обучена

=== О

## Динамическое добавление новых классов

In [ ]:
# add_new_classes.py
#from configurable_parser import ConfigurableNomenclatureParser
import joblib

def add_new_classes_dynamically():
    """Пример динамического добавления новых классов"""
    
    # Загрузка существующей модели
    try:
        parser = joblib.load('configurable_parser.pkl')
        print("Существующая модель загружена")
    except:
        parser = ConfigurableNomenclatureParser("config.yaml")
        print("Создана новая модель")
    
    # Конфигурация нового класса "арматура"
    new_class_config = {
        'тип': 'материалы',
        'характеристики': {
            'тип арматуры': {
                'тип': 'текст',
                'паттерны': ['арматура', 'фитинг', 'муфта'],
                'значения': ['отвод', 'тройник', 'муфта', 'переходник']
            },
            'диаметр': {
                'тип': 'числовая',
                'единицы': ['мм', 'дюйм'],
                'паттерны': ['диаметр', 'ø', 'размер'],
                'минимальное_значение': 15,
                'максимальное_значение': 300
            },
            'материал': {
                'тип': 'текст',
                'паттерны': ['материал', 'сталь', 'пластик'],
                'значения': ['сталь', 'ПНД', 'ПП', 'латунь']
            }
        }
    }
    
    # Добавление нового класса
    parser.add_new_class('арматура', new_class_config)
    
    # Сохранение обновленной конфигурации
    parser.save_config("config_updated.yaml")
    
    # Сохранение обновленной модели
    parser.save_model('configurable_parser_updated.pkl')
    
    print("Новый класс 'арматура' успешно добавлен")
    print(f"Характеристики: {parser.get_characteristics_for_class('арматура')}")
    
    return parser

# Тестирование нового класса
def test_new_class(parser):
    """Тестирование нового класса"""
    test_data = pd.DataFrame({
        'nomenclature': ['005'],
        'category': ['арматура'],
        'description': ['Отвод стальной диаметр 50 мм для трубопровода']
    })
    
    results = parser.process_unnormalized_data(test_data)
    
    print("\nТестирование нового класса:")
    for _, row in results.iterrows():
        print(f"Номенклатура: {row['nomenclature']}")
        print(f"Категория: {row['category']}")
        for col in results.columns:
            if col not in ['nomenclature', 'category', 'description', 'category_predicted']:
                if pd.notna(row[col]):
                    print(f"  {col}: {row[col]}")

if __name__ == "__main__":
    parser = add_new_classes_dynamically()
    test_new_class(parser)

Существующая модель загружена


AttributeError: 'dict' object has no attribute 'add_new_class'

## Процесс добавления новых классов

In [23]:
# workflow.py
def complete_workflow_for_new_classes():
    """Полный процесс добавления новых классов"""
    
    # 1. Загрузка существующей модели
    try:
        parser = EnhancedNomenclatureParser.load_model('enhanced_nomenclature_parser.pkl')
        print("Существующая модель загружена")
    except:
        parser = EnhancedNomenclatureParser()
        print("Создана новая модель")
    
    # 2. Загрузка новых данных
    new_data = pd.read_csv('новые_данные.csv')
    print(f"Загружено {len(new_data)} новых записей")
    
    # 3. Анализ новых классов
    new_categories = new_data['category'].unique()
    print(f"Новые классы: {list(new_categories)}")
    
    # 4. Автоматическое обновление
    parser = update_model_with_new_data(parser, new_data)
    
    # 5. При необходимости - ручная настройка
    if 'трубы' in new_categories:
        extended_parser = ExtendedNomenclatureParser()
        # ... перенос обученных моделей ...
    
    # 6. Тестирование
    test_results = test_new_classes(parser, new_data)
    
    # 7. Сохранение обновленной модели
    parser.save_model('nomenclature_parser_updated.pkl')
    
    return parser

def test_new_classes(parser, test_data):
    """Тестирование на новых классах"""
    print("=== ТЕСТИРОВАНИЕ НОВЫХ КЛАССОВ ===")
    
    results = parser.process_unnormalized_data(test_data)
    
    # Анализ эффективности для новых классов
    new_categories = test_data['category'].unique()
    
    for category in new_categories:
        category_data = results[results['category'] == category]
        expected_chars = parser.category_characteristics.get(category, [])
        
        print(f"\nКласс: {category}")
        print(f"Обработано записей: {len(category_data)}")
        print(f"Ожидаемые характеристики: {expected_chars}")
        
        # Анализ заполнения характеристик
        for char in expected_chars:
            if char in category_data.columns:
                fill_rate = category_data[char].notna().mean() * 100
                print(f"  {char}: {fill_rate:.1f}% заполнено")
    
    return results

## Утилиты для мониторинга и обновления

In [24]:
# monitoring.py
def monitor_model_performance(parser, new_data):
    """Мониторинг производительности модели на новых данных"""
    
    performance_report = {
        'new_classes': [],
        'characteristics_coverage': {},
        'recommendations': []
    }
    
    for category in new_data['category'].unique():
        category_data = new_data[new_data['category'] == category]
        
        if category not in parser.category_characteristics:
            performance_report['new_classes'].append(category)
            performance_report['recommendations'].append(
                f"Добавить правила извлечения для класса '{category}'"
            )
            continue
        
        # Анализ покрытия характеристик
        expected_chars = parser.category_characteristics[category]
        extracted_chars = []
        
        for desc in category_data['description'].head(10):  # Тестируем на выборке
            extracted = parser.extract_characteristics(category, desc)
            extracted_chars.extend(extracted.keys())
        
        coverage = len(set(extracted_chars)) / len(expected_chars) * 100
        performance_report['characteristics_coverage'][category] = coverage
        
        if coverage < 70:
            performance_report['recommendations'].append(
                f"Низкое покрытие характеристик для '{category}': {coverage:.1f}%"
            )
    
    return performance_report

def create_update_checklist(new_classes):
    """Создание чеклиста для обновления"""
    checklist = {
        'required_actions': [
            "Добавить новые классы в конфигурацию",
            "Определить характеристики для каждого класса", 
            "Настроить паттерны извлечения для новых характеристик",
            "Добавить специализированные методы извлечения при необходимости",
            "Протестировать на代表性тельных данных",
            "Обновить документацию"
        ],
        'new_classes': new_classes,
        'testing_requirements': [
            f"Проверить извлечение характеристик для {cls}" for cls in new_classes
        ]
    }
    
    return checklist